In [15]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
import tikzplotlib
import timeit

In [16]:
# This section is just me figuring out how tensors work
#lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
#upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
#x = tf.random.uniform(shape=[1,100]) # Interior points
#training_pts = tf.concat([lower_bound, x, upper_bound], 1) # Actual training data
#x_0 = tf.slice(training_pts, [0,0], [1,1])
#x_1 = tf.slice(training_pts, [0,101], [1,1])
#print(x_0.numpy(), x_1.numpy())

In [38]:
# Construct Simple Feedforward Network
model = Sequential()
model.add(Dense(100, input_shape=[1], activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1))
#model.compile(loss='mean_squared_error', optimizer='adam')

In [35]:
# Define Loss Function
def loss_fn(model,x):
    #print(x)
    with tf.GradientTape() as t:
        t.watch(x)
        f = model(x, training=True)
        #print('f = ', f)
    df_dx = t.gradient(f,x)
    u_0 = tf.zeros(shape=[1,1]) # Lower boundary condition
    u_1 = tf.ones(shape=[1,1]) # Upper boundary condition
    bound_weight = 100 # lambda = sqrt(bound_weight)
    mse = tf.keras.losses.MeanSquaredError()
    lower_bound_error = tf.math.subtract(tf.slice(f,[0,0],[1,1]),u_0)
    #print(lower_bound_error)
    upper_bound_error = tf.math.subtract(tf.slice(f,[99,0], [1,1]),u_1)
    loss_1 = tf.math.add(tf.nn.l2_loss(df_dx), 
                       tf.nn.l2_loss(bound_weight*lower_bound_error))
    loss = tf.math.add(loss_1, tf.nn.l2_loss(bound_weight*upper_bound_error))
    return loss

In [34]:
def train():
    # Train network
    optimizer = tf.keras.optimizers.Adam() # Fancy gradient decent
    epochs = 20
    train_loss_results = [] # For tracking loss during training
    iterations_per_epoch = 100
    minibatch_size = 100 # Number of points to be selected each iteration
    average_error = 1
    epoch = -1
    while average_error > 0.005: 
    #for epoch in range(epochs): # Uncomment to run for specific number of epochs
        epoch_loss_avg = tf.keras.metrics.Mean()
        for iteration in range(iterations_per_epoch):
            lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
            upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
            interior_pts = tf.random.uniform(shape=[1,minibatch_size-2]) # Interior points
            x = tf.transpose(tf.concat([lower_bound, interior_pts, upper_bound], 1)) # Actual training data
            with tf.GradientTape() as t:
                t.watch(x)
                #print(x)
                #f = model(x, training=True) # Estimate for u
                loss = loss_fn(model,x) # Loss
            grads = t.gradient(loss, model.trainable_weights) # Find model gradients
            optimizer.apply_gradients(zip(grads,model.trainable_weights)) # Perform gradient decent
            epoch_loss_avg.update_state(loss) # Track loss
            # End training iteration
        # Check average error    
        lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
        upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
        x = tf.sort(tf.random.uniform(shape=[1,98])) # Interior points
        test_points_tensor = tf.transpose(tf.concat([lower_bound, x, upper_bound], 1)) # Actual data
        test_points = test_points_tensor.numpy()
        g = model(test_points_tensor, training=False).numpy()
        average_error = np.sum(abs(g-test_points))/len(g)
        print(average_error)
        
        train_loss_results.append(epoch_loss_avg.result())
        epoch+=1 
        print("Epoch {:03d}: Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))
        # End Epoch

In [28]:
# Run the model with real data
train()
lower_bound = tf.zeros(shape=[1,1]) # Lower bound u(0)=0
upper_bound = tf.ones(shape=[1,1]) # Upper bound u(1)=1
x = tf.sort(tf.random.uniform(shape=[1,98])) # Interior points
test_points_tensor = tf.transpose(tf.concat([lower_bound, x, upper_bound], 1)) # Actual data
test_points = test_points_tensor.numpy()
g = model(test_points_tensor, training=False).numpy()
x_actual = tf.sort(tf.random.uniform(shape=[1,1000]).numpy()) # Known solution
#print(test_points[0])
#print(x_actual[0])
#print(f[0])
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_title('1D Laplace (DVM)')
ax.plot(x_actual[0],x_actual[0], label='u(x)', color='blue')
ax.plot(test_points,g,'r--', label='g(x)')
ax.legend()
ax.set_aspect('equal','box')

tikzplotlib.save("1d_laplace_dvm.tex")

average_error = np.sum(abs(g-test_points))/len(g)
print(average_error)

0.01731978416442871
Epoch 000: Loss: 63.748
0.01603360652923584
Epoch 001: Loss: 49.088
0.01435253381729126
Epoch 002: Loss: 49.091
0.013279361724853516
Epoch 003: Loss: 49.057
0.012523617744445801
Epoch 004: Loss: 49.043
0.01182307481765747
Epoch 005: Loss: 49.076
0.011107001304626465
Epoch 006: Loss: 49.021
0.011389490365982056
Epoch 007: Loss: 49.087
0.011728799343109131
Epoch 008: Loss: 49.190
0.013017270565032959
Epoch 009: Loss: 49.158
0.010681581497192384
Epoch 010: Loss: 49.092
0.010076498985290528
Epoch 011: Loss: 49.275
0.010208834409713746
Epoch 012: Loss: 49.403
0.009159275889396667
Epoch 013: Loss: 49.105
0.00829670250415802
Epoch 014: Loss: 49.247
0.008253355622291564
Epoch 015: Loss: 49.196
0.008552567362785339
Epoch 016: Loss: 49.258
0.010903799533843994
Epoch 017: Loss: 49.224
0.007897770404815674
Epoch 018: Loss: 49.125
0.007963371872901916
Epoch 019: Loss: 49.359
0.006701115965843201
Epoch 020: Loss: 49.139
0.006346144676208496
Epoch 021: Loss: 49.078
0.0069135373830

0.0027733880281448366
Epoch 181: Loss: 49.004
0.0036435902118682863
Epoch 182: Loss: 48.969
0.00461557388305664
Epoch 183: Loss: 49.018
0.005088064670562744
Epoch 184: Loss: 48.986
0.0030937135219573975
Epoch 185: Loss: 48.973
0.004570011496543885
Epoch 186: Loss: 60.318
0.0031241148710250853
Epoch 187: Loss: 48.968
0.003134901523590088
Epoch 188: Loss: 48.945
0.00330545037984848
Epoch 189: Loss: 48.939
0.003132765293121338
Epoch 190: Loss: 48.917
0.003526010513305664
Epoch 191: Loss: 48.985
0.0032968437671661377
Epoch 192: Loss: 48.930
0.004012938737869263
Epoch 193: Loss: 48.985
0.003410690426826477
Epoch 194: Loss: 48.997


KeyboardInterrupt: 

In [39]:
print(timeit.timeit(train, number=1))

0.005584139823913575
Epoch 000: Loss: 331.457
0.0053451406955718995
Epoch 001: Loss: 48.976
0.005005162954330444
Epoch 002: Loss: 49.014
0.004741934537887574
Epoch 003: Loss: 49.014
7.808128704999945
